In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 在GCP上进行端到端的机器学习：MLOps阶段2：实验：使用Vertex AI Training for PyTorch入门

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage2/get_started_vertex_training_pytorch.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage2/get_started_vertex_training_pytorch.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub中查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/ml_ops/stage2/get_started_vertex_training_pytorch.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI工作台中打开
    </a>
  </td>
</table>
<br/><br/><br/>

## 概览

本教程演示了如何在Google Cloud上使用Vertex AI进行端到端MLOps生产。本教程涵盖了阶段2：实验：使用PyTorch开始Vertex AI训练。

### 目标

在本教程中，您将学习如何使用`Vertex AI 训练`来训练一个 PyTorch 自定义模型。

本教程使用以下 Google Cloud ML 服务:

* `Vertex AI 训练`
* `Vertex AI 模型`资源


所执行的步骤包括:

- 使用 Python 软件包进行单节点训练。
- 在超参数调整时报告准确性。
- 使用 GCSFuse 将模型工件保存到云存储。
- 创建一个`Vertex AI 模型`资源。

### 数据集

本教程使用的数据集是来自[PyTorch数据集](https://pytorch.org/vision/stable/datasets.html#cifar)的[CIFAR10数据集](https://pytorch.org/vision/stable/datasets.html)。数据集的版本已内置于TensorFlow中。训练模型预测图像属于十个类别中的哪一类：飞机、汽车、鸟、猫、鹿、狗、青蛙、马、船或卡车。

### 成本

本教程使用 Google Cloud 的收费组件：

* Vertex AI
* Cloud Storage

了解 [Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing) 和 [Cloud Storage 价格](https://cloud.google.com/storage/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/) 根据您的预估使用量生成成本估算。

### 设置本地开发环境

**如果您正在使用Colab或Vertex AI Workbench笔记本**，您的环境已满足运行此笔记本的所有要求。您可以跳过此步骤。

否则，请确保您的环境符合此笔记本的要求。
您需要以下内容：

* Google Cloud SDK
* Git
* Python 3
* virtualenv
* 在使用Python 3的虚拟环境中运行的Jupyter笔记本

Google Cloud的[设置Python开发环境指南](https://cloud.google.com/python/setup)和[Jupyter安装指南](https://jupyter.org/install)提供了满足这些要求的详细说明。以下步骤提供了简明的一组说明：

1. [安装和初始化Cloud SDK。](https://cloud.google.com/sdk/docs/)

1. [安装 Python 3。](https://cloud.google.com/python/setup#installing_python)

1. [安装
   virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv)
   并创建一个使用Python 3的虚拟环境。激活虚拟环境。

1. 要安装Jupyter，在终端中运行`pip3 install jupyter`命令。

1. 要启动Jupyter，在终端中运行`jupyter notebook`命令。

1. 在Jupyter Notebook仪表板中打开这个笔记本。

安装以下软件包以执行此笔记本。

In [ ]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME") and not os.getenv("VIRTUAL_ENV")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

! pip3 install --upgrade google-cloud-aiplatform $USER_FLAG -q
! pip3 install --upgrade cloudml-hypertune $USER_FLAG -q
! pip3 install --upgrade torchvision $USER_FLAG -q

重新启动内核

安装了附加包之后，您需要重新启动笔记本内核才能找到这些包。

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 开始之前

### 设置您的Google Cloud项目

**无论您使用的是什么笔记本环境，以下步骤都是必需的。**

1. [选择或创建Google Cloud项目](https://console.cloud.google.com/cloud-resource-manager)。首次创建帐户时，您将获得300美元的免费信用额度来支付计算/存储费用。

1. [确保为您的项目启用了计费功能](https://cloud.google.com/billing/docs/how-to/modify-project)。

1. [启用Vertex AI、BigQuery、Compute Engine和Cloud Storage API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,bigquery,compute_component,storage_component)。

1. 如果您在本地运行此笔记本，则需要安装[Cloud SDK](https://cloud.google.com/sdk)。

1. 在下面的单元格中输入您的项目ID。然后运行该单元格，确保Cloud SDK为本笔记本中的所有命令使用正确的项目。

**注意**：Jupyter将带有`!`前缀的行视为shell命令，并将以`$`前缀的Python变量内插到这些命令中。

#### 设置您的项目ID

**如果您不知道您的项目ID**，您可以使用 `gcloud` 来获取您的项目ID。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### 区域

您还可以更改`REGION`变量，该变量用于笔记本的其余部分操作。以下是Vertex AI支持的区域。我们建议您选择离您最近的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太地区：`asia-east1`

您可能不会在使用Vertex AI进行训练时使用多地区存储桶。并非所有区域都支持所有Vertex AI服务。

了解更多关于[Vertex AI区域](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "[your-region]"  # @param {type: "string"}

if REGION == "[your-region]":
    REGION = "us-central1"

时间戳

如果您在直播教程会话中，可能会使用共享的测试账户或项目。为了避免用户之间在创建的资源上发生名称冲突，您可以为每个实例会话创建一个时间戳，并将其附加到您在本教程中创建的资源的名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### 认证您的谷歌云帐户

**如果您正在使用Vertex AI Workbench笔记本**，则您的环境已经通过验证。请跳过此步骤。

**如果您正在使用Colab**，运行下面的单元格，并按照提示进行身份验证。

**否则**，请按照以下步骤操作：

在云控制台中，转到[创建服务帐号密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey)页面。

1. **单击创建服务帐号**。

2. 在**服务帐号名称**字段中输入一个名称，然后单击**创建**。

3. 在**授予此服务帐号访问项目**部分，单击角色下拉列表。在筛选框中键入"Vertex AI"，并选择**Vertex AI管理员**。在筛选框中键入"Storage Object Admin"，并选择**存储对象管理员**。

4. 单击创建。包含您的密钥的JSON文件将下载到您的本地环境。

5. 输入您的服务帐号密钥的路径作为以下单元格中的GOOGLE_APPLICATION_CREDENTIALS变量，并运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = False
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        IS_COLAB = True
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建一个云存储桶

**无论您使用的是哪种笔记本环境，以下步骤都是必需的。**

当您初始化用于Python的Vertex SDK时，您需要指定一个云存储暂存桶。这个暂存桶是您的数据集和模型资源在不同会话中保留的地方。

在下面设置您的云存储桶的名称。存储桶的名称在所有Google Cloud项目中必须是全局唯一的，包括您机构之外的那些项目。

In [ ]:
BUCKET_URI = "gs://[your-bucket-name]"  # @param {type:"string"}

In [ ]:
if BUCKET_URI == "" or BUCKET_URI is None or BUCKET_URI == "gs://[your-bucket-name]":
    BUCKET_URI = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

只有在您的存储桶不存在时才运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_URI

最后，通过检查云存储桶的内容来验证访问权限。

In [ ]:
! gsutil ls -al $BUCKET_URI

### 设置变量

接下来，设置一些在教程中使用的变量。
### 导入库并定义常量

In [ ]:
import google.cloud.aiplatform as aiplatform

### 初始化用于 Python 的 Vertex AI SDK

为您的项目和相应的存储桶初始化 Python 版本的 Vertex AI SDK。

In [ ]:
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

#### 设置硬件加速器

您可以为训练设置硬件加速器。

设置变量 `TRAIN_GPU/TRAIN_NGPU` 以使用支持 GPU 的容器映像和分配给虚拟机（VM）实例的 GPU 数量。例如，要使用一个带有 4 个 Nvidia Telsa K80 GPU 的容器映像，分配给每个 VM，您可以指定：

    (aip.AcceleratorType.NVIDIA_TESLA_K80, 4)

否则，指定 `(None, None)` 以使用一个在 CPU 上运行的容器映像。

在这里了解更多关于您地区的硬件加速器支持。

In [ ]:
import os

if os.getenv("IS_TESTING_TRAIN_GPU"):
    TRAIN_GPU, TRAIN_NGPU = (
        aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_TRAIN_GPU")),
    )
else:
    TRAIN_GPU, TRAIN_NGPU = (aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_K80, 1)

#### 设置预构建的容器

为培训设置预构建的Docker容器镜像。

- 将变量`TF`设置为容器镜像的TensorFlow版本。例如，`2-1`表示版本2.1，`1-15`表示版本1.15。以下列表显示了一些可用的预构建映像：


有关最新列表，请参阅[用于培训的预构建容器](https://cloud.google.com/ai-platform-unified/docs/training/pre-built-containers)。

In [ ]:
if TRAIN_GPU:
    TRAIN_VERSION = "pytorch-gpu.1-9"
else:
    TRAIN_VERSION = "pytorch-xla.1-9"

TRAIN_IMAGE = "{}-docker.pkg.dev/vertex-ai/training/{}:latest".format(
    REGION.split("-")[0], TRAIN_VERSION
)

####设置机器类型

接下来，设置用于训练的机器类型。

- 将变量`TRAIN_COMPUTE`设置为配置用于训练的虚拟机的计算资源。
 - `机器类型`
     - `n1-standard`：每个vCPU有3.75GB的内存。
     - `n1-highmem`：每个vCPU有6.5GB的内存
     - `n1-highcpu`：每个vCPU有0.9GB的内存
 - `vCPUs`：数量为\[2, 4, 8, 16, 32, 64, 96 \]

*注意：以下机器类型不支持训练：*

 - `standard`：2个vCPUs
 - `highcpu`：2、4和8个vCPUs

*注意：您也可以使用n2和e2机器类型进行训练和部署，但它们不支持GPU*。

In [ ]:
if os.getenv("IS_TESTING_TRAIN_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_TRAIN_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
TRAIN_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", TRAIN_COMPUTE)

## PyTorch训练简介

PyTorch包支持单节点和分布式模型训练。

一旦您训练了一个PyTorch模型，您会想要将其保存在云存储位置，以便随后上传到`Vertex AI Model`资源。
PyTorch包不支持将模型保存到云存储位置。相反，您需要执行以下步骤将其保存到云存储位置。

1. 将内存中的模型保存到本地文件系统（例如，model.pth）。
2. 使用gsutil将本地副本复制到指定的云存储位置。

*注意*：您可以使用PyTorch模型进行超参数调整。

### 检查培训包

#### 包布局

在开始培训之前，您将查看如何为自定义培训任务组装 Python 包。解压缩后，该包包含以下目录/文件布局。

- PKG-INFO
- README.md
- setup.cfg
- setup.py
- trainer
  - \_\_init\_\_.py
  - task.py

文件 `setup.cfg` 和 `setup.py` 是将包安装到 Docker 镜像的操作环境中的指令。

文件 `trainer/task.py` 是执行自定义培训任务的 Python 脚本。*注意*，当我们在工作池规范中提到它时，我们将目录斜线替换为点号 (`trainer.task`)，并且删除文件后缀 (`.py`)。

#### 包装配

在接下来的单元格中，您将组装培训包。

In [ ]:
# Make folder for Python training script
! rm -rf custom
! mkdir custom

# Add package information
! touch custom/README.md

# Instructions for installing package into environment of the docker image
setup_cfg = "[egg_info]\n\ntag_build =\n\ntag_date = 0"
! echo "$setup_cfg" > custom/setup.cfg

setup_py = "import setuptools\n\nsetuptools.setup(\n\n    install_requires=[\n\n        'cloudml-hypertune',\n\n    ],\n\n    packages=setuptools.find_packages())"
! echo "$setup_py" > custom/setup.py

pkg_info = "Metadata-Version: 1.0\n\nName: CIFAR10 image classification\n\nVersion: 0.0.0\n\nSummary: Demostration training script\n\nHome-page: www.google.com\n\nAuthor: Google\n\nAuthor-email: aferlitsch@google.com\n\nLicense: Public\n\nDescription: Demo\n\nPlatform: Vertex"
! echo "$pkg_info" > custom/PKG-INFO

# Make the training subfolder
! mkdir custom/trainer
! touch custom/trainer/__init__.py

### 为Python培训包创建任务脚本

接下来，您将为驱动培训包创建`task.py`脚本。一些值得注意的步骤包括：

- 命令行参数：
    - `model-dir`：保存训练模型的位置。当使用Vertex AI自定义训练时，位置将在环境变量`AIP_MODEL_DIR`中指定，
    - `batch_size`/`lr`：超参数调整变量
    - `distribute`：单节点或分布式训练。
- 数据预处理（`get_data()`）：
    - 下载数据集并分割为训练集和测试集。
- 模型架构（`getmodel()`）：
    - 获取或构建模型架构。
- 训练（`train_model()`）：
    - 训练模型
- 评估（`evaluate_model()`）：
    - 评估模型。
    - 如果进行超参数调整，则报告准确度指标。
- 模型产物保存
    - 将模型产物和评估指标保存在Cloud Storage位置，由`model-dir`指定。

In [ ]:
%%writefile custom/trainer/task.py
import sys
import os
import argparse
import logging
import hypertune

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parallel import DistributedDataParallel
#import torch.backends.cudnn as cudnn
import torch.distributed as distributed
#import torch.optim
#import torch.multiprocessing as mp
import torch.utils.data as data
#import torch.utils.data.distributed
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

parser = argparse.ArgumentParser(description='PyTorch ImageNet Training')
parser.add_argument('--model-dir', dest='model_dir',
                    default=os.getenv('AIP_MODEL_DIR'), type=str, help='Model dir.')
parser.add_argument('--batch_size', dest='batch_size',
                    type=int, default=16, help='Batch size')
parser.add_argument('--epochs', dest='epochs',
                    type=int, default=20, help='Number of epochs')
parser.add_argument('--lr', dest='lr',
                    type=int, default=20, help='Learning rate')
parser.add_argument('--distribute', default="single",
                    type=str, help='Distributed training strategy')
parser.add_argument('--checkpoints', default=False,
                    type=bool, help='Whether to save checkpoints')
args = parser.parse_args()

logging.getLogger().setLevel(logging.INFO)

def distributed_is_initialized():
    if args.distribute == "mirror":
        if distributed.is_available() and distributed.is_initialized():
            return True
    return False

def get_data():

    normalize = transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    transform = transforms.Compose(
        [transforms.ToTensor(), normalize,]
    )

    train_dataset = datasets.CIFAR10(root="./train", transform=transform, train=True, download=True)
    logging.info(train_dataset)
    if distributed_is_initialized():
        sampler = data.DistributedSampler(train_dataset)
    else:
        sampler = None
    train_loader = data.DataLoader(
        train_dataset,
        batch_size=args.batch_size,
        shuffle=(sampler is None),
        sampler=sampler,
    )

    test_dataset = datasets.CIFAR10(root="./test", transform=transform, train=False, download=True)
    logging.info(test_dataset)
    sampler = None
    test_loader = data.DataLoader(
        test_dataset,
        batch_size=args.batch_size,
        shuffle=False,
        sampler=sampler,
    )

    return train_loader, test_loader

def get_model():
    class Cifar10Model(nn.Module):
        def __init__(self):
            super().__init__()
            self.conv1 = nn.Conv2d(3, 16, 3)
            self.pool = nn.MaxPool2d(2, 2)
            self.conv2 = nn.Conv2d(16, 16, 5)
            self.fc1 = nn.Linear(16 * 5 * 5, 120)
            self.fc2 = nn.Linear(120, 84)
            self.fc3 = nn.Linear(84, 10)

        def forward(self, x):
            x = self.pool(F.relu(self.conv1(x)))
            x = self.pool(F.relu(self.conv2(x)))
            x = torch.flatten(x, 1) # flatten all dimensions except batch
            x = F.relu(self.fc1(x))
            x = F.relu(self.fc2(x))
            x = self.fc3(x)
            return x

    logging.info("Get model architecture")
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    gpu_id = "0" if torch.cuda.is_available() else None
    logging.info(f"Device: {device}")

    model = Cifar10Model()
    model.to(device)

    if distributed_is_initialized():
        model = DistributedDataParallel(model)

    loss = nn.CrossEntropyLoss().cuda(gpu_id)
    optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
    return model, loss, optimizer, device

def train_model(model, loss_func, optimizer, train_loader, test_loader, is_chief, device):

    class Average(object):
        def __init__(self):
            self.sum = 0
            self.count = 0

        def __str__(self):
            return '{:.6f}'.format(self.average)

        @property
        def average(self):
            return self.sum / self.count

        def update(self, value, number):
            self.sum += value * number
            self.count += number

    class Accuracy(object):
        def __init__(self):
            self.correct = 0
            self.count = 0

        def __str__(self):
            return '{:.2f}%'.format(self.accuracy * 100)

        @property
        def accuracy(self):
            return self.correct / self.count

        @torch.no_grad()
        def update(self, output, target):
            pred = output.argmax(dim=1)
            correct = pred.eq(target).sum().item()

            self.correct += correct
            self.count += output.size(0)

    def train():

        model.train()

        train_loss = Average()
        train_acc = Accuracy()

        for data, target in train_loader:
            data = data.to(device)
            target = target.to(device)

            output = model(data)
            loss = loss_func(output, target)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss.update(loss.item(), data.size(0))
            train_acc.update(output, target)

            return train_loss, train_acc

    @torch.no_grad()
    def evaluate(epoch):
        model.eval()

        test_loss = Average()
        test_acc = Accuracy()

        for data, target in test_loader:
            data = data.to(device)
            target = target.to(device)

        output = model(data)
        loss = loss_func(output, target)

        test_loss.update(loss.item(), data.size(0))
        test_acc.update(output, target)

        # report metric for hyperparameter tuning
        hpt = hypertune.HyperTune()
        hpt.report_hyperparameter_tuning_metric(
            hyperparameter_metric_tag='accuracy',
            metric_value=test_acc.accuracy,
            global_step=epoch
        )

        return test_loss, test_acc

    for epoch in range(1, args.epochs + 1):

        logging.info('Epoch: {}, Training ...'.format(epoch))
        train_loss, train_acc = train()

        if is_chief:
            test_loss, test_acc = evaluate(epoch)

            if args.checkpoints:
                torch.save(model.state_dict(), args.model_dir + f"/{epoch}.chkpt")

            logging.info('Epoch: {}/{},'.format(epoch, args.epochs))
            logging.info('train loss: {}, train acc: {},'.format(train_loss, train_acc))
            logging.info('test loss: {}, test acc: {}.'.format(test_loss, test_acc))

    return model


train_dataset, test_dataset = get_data()
model, loss, optimizer, device = get_model()
train_model(model, loss, optimizer, train_dataset, test_dataset, True, device)

logging.info('start saving')
# export model to gcs using GCSFuse
logging.info("Exporting model artifacts ...")
gs_prefix = 'gs://'
gcsfuse_prefix = '/gcs/'
if args.model_dir.startswith(gs_prefix):
    args.model_dir = args.model_dir.replace(gs_prefix, gcsfuse_prefix)
    dirpath = os.path.split(args.model_dir)[0]
    if not os.path.isdir(dirpath):
        os.makedirs(dirpath)

gcs_model_path = os.path.join(os.path.join(args.model_dir, 'model.pth'))
torch.save(model.state_dict(), gcs_model_path)
logging.info(f'Model is saved to {args.model_dir}')

### 在本地测试训练包

接下来，使用几个时代在本地测试您完成的培训包。

In [ ]:
! python3 custom/trainer/task.py --model-dir=custom --distribute=mirror --checkpoints=True

请将培训脚本存储在您的云存储桶中

接下来，您将培训文件夹打包成压缩的tar文件，并将其存储在您的云存储桶中。

In [ ]:
! rm -f custom.tar custom.tar.gz
! tar cvf custom.tar custom
! gzip custom.tar
! gsutil cp custom.tar.gz $BUCKET_URI/trainer_cifar10.tar.gz

### 创建用于预测的 PyTorch 容器

目前，Vertex AI 没有预先定义的用于使用部署的 PyTorch 模型进行预测的容器。没问题，您可以组装自己的定制容器。通常，人们会基于 `Torch Server` 来构建容器。为了演示目的，您建立了一个占位符容器（尚未完成），其中包括最新的 `Torch Server` 图像，并将其推送到 `容器注册表`。

In [ ]:
%%writefile Dockerfile

FROM pytorch/torchserve:latest-cpu


# run Torchserve HTTP serve to respond to prediction requests
CMD ["torchserve", \n     "--start", \n     "--ts-config=/home/model-server/config.properties", \n     "--models", \n     "$APP_NAME=$APP_NAME.mar", \n     "--model-store", \n     "/home/model-server/model-store"]

In [ ]:
APP_NAME = "cifar10"
DEPLOY_IMAGE = f"gcr.io/{PROJECT_ID}/pytorch_predict_{APP_NAME}"
print(DEPLOY_IMAGE)

In [ ]:
if not IS_COLAB:
    ! docker build --tag=$DEPLOY_IMAGE ./
    ! docker push $DEPLOY_IMAGE
else:
    # install docker daemon
    ! apt-get -qq install docker.io

在Colab中执行

In [ ]:
%%bash -s $IS_COLAB $DEPLOY_IMAGE
if [ $1 == "False" ]; then
  exit 0
fi
set -x
dockerd -b none --iptables=0 -l warn &
for i in $(seq 5); do [ ! -S "/var/run/docker.sock" ] && sleep 2 || break; done
docker build --tag=$2 ./
docker push $2
kill $(jobs -p)

### 创建并运行自定义训练任务

要训练一个自定义模型，您需要执行两个步骤：1) 创建一个自定义训练任务，2) 运行这个任务。

#### 创建自定义训练任务

使用`CustomTrainingJob`类创建一个自定义训练任务，需要以下参数：

- `display_name`：自定义训练任务的可读性名称。
- `container_uri`：训练容器镜像。

- `python_package_gcs_uri`：Python训练包的位置，是一个tarball。
- `python_module_name`：Python包中训练脚本的相对路径。
- `model_serving_container_uri`：部署模型的容器镜像。

*注意：* 不需要`requirements`参数。您可以在Python包的`setup.py`脚本中指定任何依赖。

In [ ]:
DISPLAY_NAME = "cifar10_" + TIMESTAMP

job = aiplatform.CustomPythonPackageTrainingJob(
    display_name=DISPLAY_NAME,
    python_package_gcs_uri=f"{BUCKET_URI}/trainer_cifar10.tar.gz",
    python_module_name="trainer.task",
    container_uri=TRAIN_IMAGE,
    model_serving_container_image_uri=DEPLOY_IMAGE,
    project=PROJECT_ID,
)

### 准备你的命令行参数

现在为你的自定义训练容器定义命令行参数：

- `args`：传递给作为容器入口点设置的可执行文件的命令行参数。
  - `--model-dir`：对于我们的演示，我们使用这个命令行参数来指定存储模型工件的位置。
      - 直接: 将云存储位置作为命令行参数传递给训练脚本（设置变量`DIRECT=True`），或
      - 间接: 服务将云存储位置作为环境变量`AIP_MODEL_DIR`传递给你的训练脚本（设置变量`DIRECT=False`）。在这种情况下，你在作业规范中告诉服务模型工件的位置。
  - `--BLAH`：

In [ ]:
MODEL_DIR = "{}/{}".format(BUCKET_URI, TIMESTAMP)

DIRECT = False
if DIRECT:
    CMDARGS = ["--model_dir=" + MODEL_DIR]
else:
    CMDARGS = []

#### 运行自定义训练作业

接下来，您可以通过调用方法 `run` 运行自定义作业来启动训练作业，使用以下参数：

- `model_display_name`: `Model` 资源的易读名称。
- `args`: 传递给训练脚本的命令行参数。
- `replica_count`: 用于训练的计算实例数量（replica_count = 1 表示单节点训练）。
- `machine_type`: 计算实例的机器类型。
- `accelerator_type`: 硬件加速器类型。
- `accelerator_count`: 要附加到工作者副本的加速器数量。
- `base_output_dir`: 用于写入模型工件的 Cloud 存储位置。
- `sync`: 是否在作业完成之前阻塞。

In [ ]:
if TRAIN_GPU:
    model = job.run(
        model_display_name="cifar10_" + TIMESTAMP,
        args=CMDARGS,
        replica_count=1,
        machine_type=TRAIN_COMPUTE,
        accelerator_type=TRAIN_GPU.name,
        accelerator_count=TRAIN_NGPU,
        base_output_dir=MODEL_DIR,
        sync=False,
    )
else:
    model = job.run(
        model_display_name="cifar10_" + TIMESTAMP,
        args=CMDARGS,
        replica_count=1,
        machine_type=TRAIN_COMPUTE,
        base_output_dir=MODEL_DIR,
        sync=False,
    )

model_path_to_deploy = MODEL_DIR

### 列出一个定制的培训工作

In [ ]:
_job = job.list(filter=f"display_name={DISPLAY_NAME}")
print(_job)

### 等待自定义训练作业完成

接下来，等待自定义训练作业完成。或者，可以在`run()`方法中将参数`sync`设置为`True`，以阻塞直到自定义训练作业完成。

In [ ]:
model.wait()

删除自定义训练作业

当一个训练作业完成后，您可以使用 `delete()` 方法删除该训练作业。在完成之前，可以使用 `cancel()` 方法取消一个训练作业。

In [ ]:
job.delete()

清理

要清理此项目中使用的所有谷歌云资源，您可以[删除用于本教程的谷歌云项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在此教程中创建的各个资源：

- 模型
- 云存储存储桶

In [ ]:
# Delete the model using the Vertex model object
model.delete()

delete_bucket = False
if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -r $BUCKET_URI